In [ ]:
import json
import yfinance as yf

import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()


def retorna_cotacao_acao_historica(
        ticker,
        periodo='1mo'
):
    ticker_obj = yf.Ticker(f'{ticker}.SA')
    hist = ticker_obj.history(period=periodo)['Close']
    hist.index = hist.index.strftime('%Y-%m-%d')
    hist = round(hist, 2)
    if len(hist) > 30:
        slice_size = int(len(hist) / 30)
        hist = hist.iloc[::-slice_size][::-1]
    return hist.to_json()


tools = [
    {
        'type': 'function',
        'function': {
            'name': 'retorna_cotacao_acao_historica',
            'description': 'Retorna a cotação diária histórica para uma ação da bovespa',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3" para ambev, "PETR4" para petrobras, etc'
                    },
                    'periodo': {
                        'type': 'string',
                        'description': 'O período que será retornado de dados históriocos \
                                        sendo "1mo" equivalente a um mês de dados, "1d" a \
                                        1 dia e "1y" a 1 ano',
                        'enum': ["1d","5d","1mo","6mo","1y","5y","10y","ytd","max"]
                    }
                }
            }
        }
    }

]

funcoes_disponiveis = {'retorna_cotacao_acao_historica': retorna_cotacao_acao_historica}


mensagens = [{'role': 'user', 'content': 'Qual é a cotação da ambev agora'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    tools=tools,
    tool_choice='auto'
)


In [ ]:
tool_calls = resposta.choices[0].message.tool_calls

if tool_calls:
    mensagens.append(resposta.choices[0].message)
    for tool_call in tool_calls:
        func_name = tool_call.function.name
        function_to_call = funcoes_disponiveis[func_name]
        func_args = json.loads(tool_call.function.arguments)
        func_return = function_to_call(**func_args)
        mensagens.append({
            'tool_call_id': tool_call.id,
            'role': 'tool',
            'name': func_name,
            'content': func_return
        })
    segunda_resposta = client.chat.completions.create(
        messages=mensagens,
        model='gpt-3.5-turbo-0125',
    )
    mensagens.append(segunda_resposta.choices[0].message)


In [ ]:
segunda_resposta.choices[0].message.content

'A cotação da ação da Ambev (ABEV3) agora é de R$ 12,54.'

In [ ]:
func_name = tool_call.function.name
function_to_call = funcoes_disponiveis[func_name]
func_args = json.loads(tool_call.function.arguments)
func_return = function_to_call(**func_args)


func_return

'{"2024-03-20":12.54}'

In [ ]:
func_args

{'ticker': 'ABEV3', 'periodo': '1d'}

In [ ]:
tool_call

ChatCompletionMessageToolCall(id='call_VDtRP7mzpaqlvklVZYkQc5jS', function=Function(arguments='{"ticker":"ABEV3","periodo":"1d"}', name='retorna_cotacao_acao_historica'), type='function')

# Descrição do Código e do Contexto de Aprendizado

## **O que é:**
O código apresentado é uma implementação de um chatbot financeiro que interage com o usuário e utiliza a API da OpenAI para gerar respostas dinâmicas. O chatbot é capaz de processar a entrada do usuário, chamar funções externas (como obter a cotação histórica de uma ação), e integrar essas respostas às conversas geradas pelo modelo GPT.

## **O que está sendo usado:**

#### Bibliotecas Importadas:

 - **openai:** Interage com os modelos da OpenAI.
 - **yfinance:** Obtém dados financeiros, como cotações de ações.
 - **dotenv:** Gerencia variáveis de ambiente.
 - **json:** Trabalha com dados no formato JSON.

#### Variáveis de Ambiente:

A biblioteca dotenv é usada para carregar uma chave de API de um arquivo .env, evitando a exposição direta da chave no código-fonte.

#### Função retorna_cotacao_acao_historica:

Função que obtém o histórico de cotações de uma ação da Bovespa e retorna os dados no formato JSON. Essa função pode ser chamada dinamicamente pelo modelo GPT durante uma conversa.

#### API da OpenAI:

A API da OpenAI é usada para gerar texto com base em mensagens trocadas com o usuário. O código utiliza um modelo da série GPT-3.5 para processar a conversa.

## **Para que serve:**

O objetivo deste código é simular um chatbot financeiro que pode gerar respostas complexas e utilizar ferramentas externas (como a função de cotação de ações) em tempo real. Ele pode ser utilizado para obter informações financeiras e integrar essas respostas com a capacidade de linguagem do modelo GPT.

## **Em que momento do aprendizado estamos:**

Estamos no ponto de explorar como integrar funções externas com a API da OpenAI. O foco é demonstrar como utilizar o modelo GPT para gerar respostas dinâmicas e, ao mesmo tempo, habilitar a chamada de funções personalizadas dentro dessas respostas, como a obtenção de dados financeiros.